In [46]:
import numpy as np
import os

In [47]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [48]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        rng = np.random.default_rng(seed=0)
        self.w = rng.normal(size=(dim, 1)) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        
        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=1000, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        assert len(x) == len(y), \
            "Количество экземпляров в массиве X не равно количеству меток в массиве Y. " + \
            f"Полученные размеры: len(X) = {len(x)}, len(Y) = {len(y)}."
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
        return self

## 1. Применение логистической регрессии (несбалансированные данные)

### 1.1 Создание и обучение логистической регрессии

In [49]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score, precision_score
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')


In [50]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.


# Создание модели логистической регрессии
model = LogisticRegression(dim=x_train.shape[1])

# Обучение модели
model.fit(x_train, y_train)

In [51]:
# Получите предсказания на тестовой выборке и оцените точность модели, 
# используя accuracy_score из пакета SciKit-Learn.
# Получение предсказаний на тестовой выборке
y_pred = model.predict(x_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.9045454545454545


### 1.2 Анализ качества модели

In [52]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`. 

class DummyClassifier:
    def __init__(self):
        print('Hello, brother!')
        
    def predict(self, x):
        # Всегда предсказывает класс 0
        return np.zeros(x.shape[0], dtype=int)
    
dummy_classifier = DummyClassifier()
y_dummy_pred = dummy_classifier.predict(x_test)

Hello, brother!


In [53]:
# Оцените точность "глупого классификатора", объясните результат.
dummy_accuracy = accuracy_score(y_test, y_dummy_pred)
print(f'Dummy Classifier Accuracy: {dummy_accuracy}')

Dummy Classifier Accuracy: 0.9090909090909091


"Глупый классификатор" всегда предсказывает класс 0, поэтому его точность будет зависеть от того, сколько экземпляров класса 0 в тестовом наборе.

In [54]:
# Используйте дополнительные метрики (f1-score, recall, precision) из пакета sklearn для анализа "глупого классификатора".
dummy_f1 = f1_score(y_test, y_dummy_pred, average='binary')
dummy_recall = recall_score(y_test, y_dummy_pred, average='binary')
dummy_precision = precision_score(y_test, y_dummy_pred, average='binary')

print(f'Dummy Classifier f1: {dummy_f1}')
print(f'Dummy Classifier recall: {dummy_recall}')
print(f'Dummy Classifier precision: {dummy_precision}')

C:\Users\Lubus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [55]:
# Используя те же метрики, проанализируйте обученную вами модель логистической регрессии.
model_f1 = f1_score(y_test, y_pred, average='binary')
model_recall = recall_score(y_test, y_pred, average='binary')
model_precision = precision_score(y_test, y_pred, average='binary')

In [56]:
# Объясните результат, описав его комментариями в этой клетке.

Логистическая регрессия, обученная на несбалансированных данных, может давать хорошие результаты по метрике точности, но метрики F1, Recall и Precision могут показать проблемы с классификацией менее представленных классов.

### 1.3 Анализ набора данных

In [57]:
# Посчитайте количество экземпляров данных для каждого класса.
unique, counts = np.unique(y_train, return_counts=True)
print(f"Class distribution in training set: {dict(zip(unique, counts))}")


Class distribution in training set: {0.0: 200, 1.0: 20}


In [58]:
# Предложите способ улучшения качества модели. Подсказка: добавление дубликатов в данные.
# Указание: не изменяйте тестовую выборку.

# Увеличение данных для класса с меньшим количеством экземпляров
class_0_count = counts[0]
class_1_count = counts[1]

# Если класс 1 меньше, увеличим его до класса 0
if class_1_count < class_0_count:
    diff = class_0_count - class_1_count
    indices_class_1 = np.where(y_train == 1)[0]
    x_train_upsampled = np.vstack([x_train, x_train[indices_class_1][:diff]])
    y_train_upsampled = np.hstack([y_train, y_train[indices_class_1][:diff]])
else:
    x_train_upsampled = x_train
    y_train_upsampled = y_train

In [59]:
# Создайте и обучите модель с использованием предложенных наработок.
model_balanced = LogisticRegression(dim=x_train.shape[1])
model_balanced.fit(x_train_upsampled, y_train_upsampled)

In [60]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь сбалансировать данные таким образом, чтобы новая модель была ощутимо лучше старой.

y_pred_balanced = model_balanced.predict(x_test)

# Оценка метрик для новой модели
balanced_accuracy = accuracy_score(y_test, y_pred_balanced)
balanced_f1 = f1_score(y_test, y_pred_balanced, average='binary')
balanced_recall = recall_score(y_test, y_pred_balanced, average='binary')
balanced_precision = precision_score(y_test, y_pred_balanced, average='binary')

print(f'Balanced Model Accuracy: {balanced_accuracy}')
print(f'Balanced Model F1 Score: {balanced_f1}')
print(f'Balanced Model Recall: {balanced_recall}')
print(f'Balanced Model Precision: {balanced_precision}')

Balanced Model Accuracy: 0.9136363636363637
Balanced Model F1 Score: 0.4864864864864865
Balanced Model Recall: 0.45
Balanced Model Precision: 0.5294117647058824


В данном решении мы сделали данные более сбалансированными, что привело к улучшению метрик для класса с меньшим количеством примеров.

## 2. Применение логистической регрессии (нелинейные данные)

In [61]:
x_train, y_train, x_test, y_test = load_data('dataset2')

In [62]:
# Создайте и обучите модель но этом наборе данных.
model = LogisticRegression(dim=x_train.shape[1])

# Обучение модели на исходных данных
model.fit(x_train, y_train)

In [63]:
# Получение предсказаний на тестовой выборке
y_pred = model.predict(x_test)

# Оценка качества модели

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
precision = precision_score(y_test, y_pred, average='binary')

print(f'Initial Model Accuracy: {accuracy}')
print(f'Initial Model F1 Score: {f1}')
print(f'Initial Model Recall: {recall}')
print(f'Initial Model Precision: {precision}')

Initial Model Accuracy: 0.57
Initial Model F1 Score: 0.6194690265486725
Initial Model Recall: 0.7777777777777778
Initial Model Precision: 0.5147058823529411


Изначальная модель показывает базовые результаты на нелинейных данных. В таких случаях линейная модель может не справляться с задачей классификации, так как границы между классами могут быть сложными и нелинейными.

In [64]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, ...).
# Объедините трансформированные данные с исходными (важно: количество экземпляров в x_train не должно увеличиться).
# Применение нелинейных функций на данных
x_train_sin = np.sin(x_train)
x_train_tanh = np.tanh(x_train)

x_test_sin = np.sin(x_test)
x_test_tanh = np.tanh(x_test)

# Объединение оригинальных и трансформированных данных
x_train_augmented = np.hstack((x_train, x_train_sin, x_train_tanh))
x_test_augmented = np.hstack((x_test, x_test_sin, x_test_tanh))

In [65]:
# Создайте и обучите модель с использованием наработок.
# Обучение новой модели на расширенных данных
model_augmented = LogisticRegression(dim=x_train_augmented.shape[1])

model_augmented.fit(x_train_augmented, y_train, iters=5000, lr=0.001)

# Получение предсказаний на расширенной тестовой выборке
y_pred_augmented = model_augmented.predict(x_test_augmented)


In [66]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь добиться точности в 100%!
# Оценка новой модели
augmented_accuracy = accuracy_score(y_test, y_pred_augmented)
augmented_f1 = f1_score(y_test, y_pred_augmented, average='binary')
augmented_recall = recall_score(y_test, y_pred_augmented, average='binary')
augmented_precision = precision_score(y_test, y_pred_augmented, average='binary')

print(f'Augmented Model Accuracy: {augmented_accuracy}')
print(f'Augmented Model F1 Score: {augmented_f1}')
print(f'Augmented Model Recall: {augmented_recall}')
print(f'Augmented Model Precision: {augmented_precision}')

Augmented Model Accuracy: 0.86
Augmented Model F1 Score: 0.8157894736842105
Augmented Model Recall: 0.6888888888888889
Augmented Model Precision: 1.0


Feature Engineering с использованием нелинейных функций (таких как sin и tanh) помогает добавлять новые признаки, которые позволяют модели лучше разделять данные. Это может улучшить качество классификации.

По сравнению с исходной моделью, модель с расширенными данными должна показать лучшее качество на тестовом наборе данных.

## 3. Доп. задания (любое на выбор, опционально)

### 3.1 'Упрощение' логистической регрессии

Сложность: легко.

In [67]:
"""
Модифицируйте класс логистической регрессии так, чтобы в нём не использовалась сигмоида.
То есть вывод о предсказанном классе должен делаться на основе значений "до сигмоиды".
Вспомогательная ссылка: https://en.wikipedia.org/wiki/Logit
"""

class LogisticRegression:
    def __init__(self, dim=2):
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        p = sigmoid(x)
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=1000, lr=0.01):
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)

In [68]:
# Перенесите обученные веса модели из пункта 1.3 в новую модель с модифицированным кодом


In [69]:
# Убедитесь, что предсказания модели с модифицированными кодом совпадают с предсказаниями
# модели из пункта 1.3

### 3.2 'Обобщение' логистической регрессии

Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.

In [70]:
x_train, y_train, x_test, y_test = load_data('dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.

In [71]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий, 
где `C` - количество классов. i-ая логистическая регрессия производит 
бинарную классификацию вида: все остальные классы и i-ый класс.
"""

class MulticlassLogisticRegression:
    def __init__(self, n_classes, dim):
        """
        n_classes - количество классов
        dim - размерность входных данных
        """
        self.n_classes = n_classes
        self.models = []
        
        # Инициализация логистических регрессий для каждого класса
        for _ in range(n_classes):
            self.models.append(LogisticRegression(dim))
    
    def predict(self, x):
        """
        x - numpy массив размерности [N, dim]
        Возвращает массив целых чисел размерности [N], где i-ый элемент обозначает номер класса
        для i-го экземпляра данных в `x`.
        """
        # Получаем вероятности для каждого класса
        probs = np.zeros((x.shape[0], self.n_classes))
        
        for i, model in enumerate(self.models):
            probs[:, i] = model.predict(x, probs=True).flatten()
        
        # Возвращаем класс с максимальной вероятностью
        return np.argmax(probs, axis=1)
    
    def fit(self, x, y, iters=1000, lr=0.01):
        """
        x - numpy массив входных данных [N, dim]
        y - numpy массив меток классов [N]
        iters - количество итераций для обучения
        lr - скорость обучения
        """
        for i in range(self.n_classes):
            # Для i-го класса создаем бинарные метки: 1 для текущего класса, 0 для всех остальных
            y_binary = (y == i).astype(int)
            self.models[i].fit(x, y_binary, iters=iters, lr=lr)

In [72]:
# Создайте и обучите написанный классификатор. Оцените точность модели.
# Загрузка данных
x_train, y_train, x_test, y_test = load_data('dataset3')

# Определение количества классов на основе уникальных значений в y_train
n_classes = len(np.unique(y_train))

# Создание и обучение модели
model = MulticlassLogisticRegression(n_classes, x_train.shape[1])
model.fit(x_train, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(x_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Multiclass Logistic Regression Accuracy: {accuracy * 100:.2f}%')

Multiclass Logistic Regression Accuracy: 94.67%


1. Инициализация: В конструкторе MulticlassLogisticRegression создается по одному экземпляру логистической регрессии для каждого класса. Все модели обучаются независимо.
2. Обучение: Для каждого класса мы создаем бинарные метки, где метка класса равна 1, а все остальные метки равны 0. Таким образом, каждая логистическая регрессия обучается на задаче "один против всех".
3. Предсказание: Для каждого экземпляра данных мы получаем вероятность от каждой модели и выбираем класс с наибольшей вероятностью.
4. Оценка: Используем метрику точности для оценки качества модели.

<b>Softmax классификатор.</b> Сложность: математический гений.

In [73]:
"""
Напишите класс классификатора, основанного на функции Softmax.
Алгоритм работы данного классификатора:
x - вектор (экземпляр данных) размерности dim.
W - матрица весов размерности [dim, n_classes].

Ответ классификатора формируется как:
logits = x * W - матричное умножение
p = softmax(logits)
class_id = argmax(p)

Для данного классификатора требуется модифицировать алгоритм обучения в методе fit.

Вспомогательные ресурсы:
https://en.wikipedia.org/wiki/Softmax_function
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
"""

class SoftmaxClassificator:
    def __init__(self, n_classes, dim):
        pass
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        pass
    
    def fit(self, x, y):
        pass

In [74]:
# Создайте и обучите написанный классификатор. Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).


In [75]:
# Создайте и обучите написанный классификатор на наборе данных из задания 1 (опционально). 
# Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).
